<a href="https://colab.research.google.com/github/setiawansandi/image_recognition/blob/main/mlai_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
while True: pass